# Imports

In [ ]:
import numpy as np
import pandas as pd
import zarr
import dask
from dask import delayed
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
import streamz
import streamz.dataframe as sdf
import holoviews as hv
from holoviews.streams import Stream, param
from holoviews.operation.datashader import regrid
from bokeh.models.tools import HoverTool
import matplotlib.pyplot as plt
import qgrid
import ipywidgets as widgets
from tqdm import tnrange, tqdm, tqdm_notebook
import warnings
from functools import partial
from cytoolz import *
from operator import getitem
import nd2reader
from importlib import reload
import traceback
import hvplot.pandas
import param
import parambokeh
from traitlets import All
import cachetools
from collections import namedtuple, defaultdict
from collections.abc import Mapping, Sequence
from numbers import Number
import skimage.morphology
import scipy
from glob import glob

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common, trench_detection, util
import ui, diagnostics, metadata
import workflow, image, geometry
import trench_detection.hough, trench_detection.core
import trench_segmentation.watershed

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Restore data

In [ ]:
%store -r trench_points
%store -r trench_diag
%store -r trench_bboxes

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="00:30:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="8GB",
    local_directory="/tmp",
    cores=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
cluster.stop_jobs(cluster.running_jobs.keys())

In [ ]:
# client = Client()

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2',
#                 '/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2', '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']
nd2_filenames = glob("/n/scratch2/jqs1/fidelity/all/180405_*.nd2") + glob(
    "/n/scratch2/jqs1/fidelity/all/TrErr*.nd2"
)

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)
image_limits = workflow.get_filename_image_limits(metadata)

# Reload

In [ ]:
def do_reload():
    from importlib import reload
    import util, trench_detection, diagnostics, workflow, image

    # reload(util)
    reload(trench_detection.hough)
    # reload(diagnostics)
    # reload(workflow)
    # reload(image)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :, ["MCHERRY"], 0], :]

In [ ]:
len(frames_to_process)

## Run trench finding

In [ ]:
%%time
find_trenches_diag = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=True, pandas=True
)
trench_info_futures = {
    idx: client.submit(
        find_trenches_diag, client.submit(workflow.get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
client.cancel(trench_info_futures)

In [ ]:
def as_completed(obj, with_results=True):
    if isinstance(obj, Mapping):
        futures = obj.values()
        dask_to_keys = {future.key: k for k, future in obj.items()}
    else:
        raise NotImplementedError
    for res in distributed.as_completed(futures, with_results=with_results):
        if with_results:
            future, result = res
            yield dask_to_keys[future.key], future, result
        else:
            future = res
            yield dask_to_keys[future.key], future

In [ ]:
trench_info = {}
for key, fut, res in as_completed(trench_info_futures):
    trench_info[key] = res
    client.cancel(fut)

In [ ]:
progress(trench_info_futures)

In [ ]:
%%time
trench_info = util.apply_map_futures(
    client.gather, trench_info_futures, predicate=lambda x: x.status == "error"
)

In [ ]:
%%time
%store trench_info

In [ ]:
len(trench_info)

In [ ]:
len(errs)

In [ ]:
errs = {k: v[2] for k, v in trench_info.items() if v[2] is not None}
errs

In [ ]:
%%time
trench_points, trench_diag, trench_err = workflow.unzip_trench_info(trench_info)

In [ ]:
len(trench_points)

In [ ]:
%%time
%store trench_points
%store trench_diag

## Analysis

In [ ]:
bad_angle = trench_diag["find_trench_lines.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_pitch = (trench_diag["find_trench_lines.hough_2.peak_func.pitch"] - 24).abs() > 1
bad_pitch.sum()

In [ ]:
selected = trench_diag[bad_pitch]  # trench_diag[bad_angle | bad_period]

In [ ]:
frame_stream.event(_df=selected.index.to_frame(index=False))

In [ ]:
%%time
trench_points_good = trench_points[~util.multi_join(trench_points.index, bad_pitch)]

In [ ]:
(len(trench_points_good), len(trench_points_good) / len(trench_points))

In [ ]:
%%time
trench_bbox_futures = []
for _, trenches in trench_points_good.groupby(["filename", "position", "t"]):
    trench_bbox_futures.append(
        client.submit(workflow.get_trench_bboxes, trenches, image_limits)
    )

In [ ]:
%%time
trench_bbox_results = util.apply_map_futures(
    client.gather, trench_bbox_futures, predicate=lambda x: x.status == "finished"
)
trench_bboxes = pd.concat(
    [trench_points_good, pd.concat(trench_bbox_results, axis=0)], axis=1
)

In [ ]:
%%time
%store trench_bboxes

In [ ]:
%store -r trench_bboxes

In [ ]:
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]
# trench_bboxes_t0.index = trench_points_good_t0.index.droplevel('t')

# Trench finding QA

In [ ]:
selected = all_frames

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

In [ ]:
frame = workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))

In [ ]:
tp, diag, _ = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=False
)(frame_rot)

In [ ]:
ui.show_plot_browser(diag)

# Segmentation

In [ ]:
selected_trenches_segmentation = trench_bboxes[
    trench_bboxes[("info", "hough_value")] > 90
].loc[IDX[:, :3, ["MCHERRY"], 0, :, :], :]
# selected_trenches_index = next(iter(selected_trenches_segmarker.groupby('t')))[1].index.droplevel('t')
# selected_trenches_reporter = selected_trenches_segmarker.rename(index={'MCHERRY': 'YFP'}, level='channel', copy=False)
# selected_trenches_all = pd.concat([selected_trenches_segmarker, selected_trenches_reporter]).sort_index()

In [ ]:
selected_trenches_index.to_frame().head()

In [ ]:
selected_trenches_segmarker.head()

In [ ]:
fpt_stream = streamz.Stream()

In [ ]:
for frame_idx, trenches in take(
    2,
    util.iter_index(
        selected_trenches_index.to_series().groupby(["filename", "position"])
    ),
):
    # display(selected_trenches_all.xs((frame_idx.filename,frame_idx.position), drop_level=False).tail())
    display(trenches.to_frame().tail())
    print(frame_idx)

In [ ]:
def do_segment_trench(img_stack):
    label_stack = np.stack([segment_trench(img) for img in img_stack])
    return zarr.array(
        label_stack,
        compressor=Blosc(cname="zstd", clevel=5, shuffle=Blosc.NOSHUFFLE, blocksize=0),
    )

In [ ]:
[all_frames_future] = client.scatter([all_frames], broadcast=True)

In [ ]:
a = all_frames_t.get_group(
    ("/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 0)
).groupby("t")

In [ ]:
a.groups.keys()

In [ ]:
all_frames_t.groups[("/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2", 0)]

In [ ]:
labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.1", "p0.3", "p0.5", "p0.7", "p0.9"): partial(
        np.percentile, q=(0.1, 0.3, 0.5, 0.7, 0.9)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

segmentation_channel = "MCHERRY"
reporter_channels = ["MCHERRY", "YFP"]

all_frames_t = all_frames.groupby(["filename", "position"])
# for frame_idx, trenches in take(2, util.iter_index(selected_trenches_all.groupby(['filename', 'position']))):
# for frame_idx, trenches in take(2, util.iter_index(selected_trenches_index.to_series().groupby(['filename', 'position']))):
position_trenchwise_dfs = {}
position_labelwise_dfs = {}
for frame_idx, trenches in take(
    2, util.iter_index(selected_trenches_segmentation.groupby(["filename", "position"]))
):
    # display(trenches)
    uls = trenches["upper_left"].values
    lrs = trenches["lower_right"].values
    trench_idxs = trenches.index.get_level_values("trench")
    # get timepoints/channels for this position
    fp_frames = all_frames_t.get_group((frame_idx.filename, frame_idx.position))
    position_t_trenchwise_dfs = {}
    position_t_labelwise_dfs = {}
    for t, frames in take(2, fp_frames.groupby("t")):
        # TODO: make sure frames contains MCHERRY, YFP
        channels = {segmentation_channel, *reporter_channels}
        images = {
            channel: workflow.get_nd2_frame(channel=channel, t=t, **frame_idx._asdict())
            for channel in channels
        }
        trench_trenchwise_dfs = {}
        trench_labelwise_dfs = {}
        for trench_idx in take(2, trench_idxs):
            ul = uls[trench_idx]
            lr = lrs[trench_idx]
            trench_images = {
                channel: images[channel][ul[1] : lr[1] + 1, ul[0] : lr[0] + 1]
                for channel in channels
            }
            trench_trenchwise_df = pd.concat(
                {
                    channel: workflow.map_frame(
                        trenchwise_funcs, trench_images[channel]
                    )
                    for channel in reporter_channels
                },
                axis=0,
            )
            trench_trenchwise_dfs[trench_idx] = trench_trenchwise_df
            label_trench_image = trench_segmentation.watershed.segment_trench(
                trench_images[segmentation_channel]
            )
            trench_labelwise_df = pd.concat(
                {
                    channel: pd.concat(
                        {
                            "labelwise": workflow.map_frame_over_labels(
                                funcs, label_trench_image, trench_images[channel]
                            ),
                            "regionprops": image.regionprops(
                                label_trench_image, trench_images[channel]
                            ),
                        },
                        axis=1,
                    )
                    for channel in reporter_channels
                },
                axis=1,
            )
            trench_labelwise_dfs[trench_idx] = trench_labelwise_df
        position_t_trenchwise_df = pd.concat(trench_trenchwise_dfs, axis=1).T
        position_t_trenchwise_df.index.name = "trench"
        position_t_trenchwise_dfs[t] = position_t_trenchwise_df
        position_t_labelwise_df = pd.concat(trench_labelwise_dfs, axis=0)
        position_t_labelwise_df.index.set_names("trench", level=0, inplace=True)
        position_t_labelwise_dfs[t] = position_t_labelwise_df
    position_trenchwise_df = pd.concat(position_t_trenchwise_dfs, axis=0)
    position_trenchwise_df.index.set_names("t", level=0, inplace=True)
    position_trenchwise_dfs[tuple(frame_idx)] = position_trenchwise_df
    position_labelwise_df = pd.concat(position_t_labelwise_dfs, axis=0)
    position_labelwise_df.index.set_names("t", level=0, inplace=True)
    position_labelwise_dfs[tuple(frame_idx)] = position_labelwise_df
trenchwise_df = pd.concat(position_trenchwise_dfs, axis=0)
trenchwise_df.index.names = ["filename", "position", *trenchwise_df.index.names[2:]]
labelwise_df = pd.concat(position_labelwise_dfs, axis=0)
labelwise_df.index.names = ["filename", "position", *labelwise_df.index.names[2:]]
display(trenchwise_df)
display(labelwise_df)

# Stream segmentation

In [ ]:
labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.1", "p0.3", "p0.5", "p0.7", "p0.9"): partial(
        np.percentile, q=(0.1, 0.3, 0.5, 0.7, 0.9)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

segmentation_channel = "MCHERRY"
reporter_channels = ["MCHERRY", "YFP"]


def per_trench(trench_idx, uls, lrs, channels):
    ul = uls[trench_idx]
    lr = lrs[trench_idx]
    trench_images = {
        channel: images[channel][ul[1] : lr[1] + 1, ul[0] : lr[0] + 1]
        for channel in channels
    }
    trench_trenchwise_df = pd.concat(
        {
            channel: workflow.map_frame(trenchwise_funcs, trench_images[channel])
            for channel in reporter_channels
        },
        axis=0,
    )
    return trench_trenchwise_df


def finalize_trench(trench_trenchwise_dfs):
    position_t_trenchwise_df = pd.concat(trench_trenchwise_dfs, axis=1).T
    position_t_trenchwise_df.index.name = "trench"
    return position_t_trenchwise_df


def finalize_position_t(position_t_trenchwise_df):
    position_trenchwise_df = pd.concat(position_t_trenchwise_dfs, axis=0)
    position_trenchwise_df.index.set_names("t", level=0, inplace=True)
    return position_trenchwise_df


def finalize(position_trenchwise_dfs):
    trenchwise_df = pd.concat(position_trenchwise_dfs, axis=0)
    trenchwise_df.index.names = ["filename", "position", *trenchwise_df.index.names[2:]]
    return trenchwise_df


all_frames_t = all_frames.groupby(["filename", "position"])
position_trenchwise_dfs = {}
for frame_idx, trenches in take(
    2, util.iter_index(selected_trenches_segmentation.groupby(["filename", "position"]))
):
    uls = trenches["upper_left"].values
    lrs = trenches["lower_right"].values
    trench_idxs = trenches.index.get_level_values("trench")
    fp_frames = all_frames_t.get_group((frame_idx.filename, frame_idx.position))
    position_t_trenchwise_dfs = {}
    position_t_labelwise_dfs = {}
    for t, frames in take(2, fp_frames.groupby("t")):
        channels = {segmentation_channel, *reporter_channels}
        images = {
            channel: workflow.get_nd2_frame(channel=channel, t=t, **frame_idx._asdict())
            for channel in channels
        }
        trench_trenchwise_dfs = {}
        for trench_idx in take(2, trench_idxs):
            trench_trenchwise_dfs[trench_idx] = per_trench(
                trench_idx, uls, lrs, channels
            )
        position_t_trenchwise_dfs[t] = finalize_trench(trench_trenchwise_dfs)
    position_trenchwise_dfs[tuple(frame_idx)] = finalize_position_t(
        position_t_trenchwise_dfs
    )
trenchwise_df = finalize(position_trenchwise_dfs)
display(trenchwise_df)

In [ ]:
labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.1", "p0.3", "p0.5", "p0.7", "p0.9"): partial(
        np.percentile, q=(0.1, 0.3, 0.5, 0.7, 0.9)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

segmentation_channel = "MCHERRY"
reporter_channels = ["MCHERRY", "YFP"]


def per_trench(trench_idx, uls, lrs, images, channels, reporter_channels=None):
    ul = uls[trench_idx]
    lr = lrs[trench_idx]
    trench_images = {
        channel: images[channel][ul[1] : lr[1] + 1, ul[0] : lr[0] + 1]
        for channel in channels
    }
    trench_trenchwise_df = pd.concat(
        {
            channel: workflow.map_frame(trenchwise_funcs, trench_images[channel])
            for channel in reporter_channels
        },
        axis=0,
    )
    return trench_trenchwise_df


def per_t(frame_idx, t, trench_idxs, uls, lrs, channels=None, per_trench_func=None):
    images = {
        channel: workflow.get_nd2_frame(channel=channel, t=t, **frame_idx._asdict())
        for channel in channels
    }
    trench_dfs = {}
    for trench_idx in take(2, trench_idxs):
        trench_dfs[trench_idx] = per_trench_func(trench_idx, uls, lrs, images, channels)
    df = pd.concat(trench_dfs, axis=1).T
    df.index.name = "trench"
    return df


def finalize(dfs):
    trenchwise_df = pd.concat(dfs, axis=0)
    trenchwise_df.index.names = [
        "filename",
        "position",
        "t",
        *trenchwise_df.index.names[3:],
    ]
    return trenchwise_df


channels = {segmentation_channel, *reporter_channels}
per_t_func = partial(
    per_t,
    channels=channels,
    per_trench_func=partial(per_trench, reporter_channels=reporter_channels),
)

all_frames_t = all_frames.groupby(["filename", "position"])
dfs = {}
for frame_idx, trenches in take(
    2, util.iter_index(selected_trenches_segmentation.groupby(["filename", "position"]))
):
    uls = trenches["upper_left"].values
    lrs = trenches["lower_right"].values
    trench_idxs = trenches.index.get_level_values("trench")
    fp_frames = all_frames_t.get_group((frame_idx.filename, frame_idx.position))
    for t, frames in take(2, fp_frames.groupby("t")):
        dfs[(*frame_idx, t)] = per_t_func(frame_idx, t, trench_idxs, uls, lrs)
trenchwise_df = finalize(dfs)
display(trenchwise_df)

In [ ]:
def _analyze_trench(
    trench_images,
    channels=None,
    trenchwise_funcs=None,
    labelwise_funcs=None,
    regionprops=True,
):
    if trenchwise_funcs:
        trenchwise_df = pd.concat(
            {
                channel: workflow.map_frame(trenchwise_funcs, trench_images[channel])
                for channel in reporter_channels
            },
            axis=0,
        )
    else:
        trenchwise_df = None
    if labelwise_funcs or regionprops:
        label_trench_image = trench_segmentation.watershed.segment_trench(
            trench_images[segmentation_channel]
        )
        labelwise_df = pd.concat(
            {
                channel: pd.concat(
                    {
                        "labelwise": workflow.map_frame_over_labels(
                            funcs, label_trench_image, trench_images[channel]
                        ),
                        "regionprops": image.regionprops(
                            label_trench_image, trench_images[channel]
                        ),
                    },
                    axis=1,
                )
                for channel in reporter_channels
            },
            axis=1,
        )
    else:
        labelwise_df = None
    return trenchwise_df, labelwise_df


def _get_trench_crops(images, trench_idx, uls, lrs):
    ul = uls[trench_idx]
    lr = lrs[trench_idx]
    trench_images = {
        channel: images[channel][ul[1] : lr[1] + 1, ul[0] : lr[0] + 1]
        for channel in images.keys()
    }
    return trench_images


def analyze_trenches(
    frame_idx,
    t,
    trenches,
    channels=None,
    framewise_funcs=None,
    trenchwise_funcs=None,
    labelwise_funcs=None,
    regionprops=True,
):
    trench_idxs = trenches.index.get_level_values("trench")
    uls = trenches["upper_left"].values
    lrs = trenches["lower_right"].values
    images = {
        channel: workflow.get_nd2_frame(channel=channel, t=t, **frame_idx._asdict())
        for channel in channels
    }
    if framewise_funcs:
        framewise_df = pd.concat(
            {
                channel: workflow.map_frame(framewise_funcs, images[channel])
                for channel in reporter_channels
            },
            axis=0,
        )
    else:
        framewise_df = None
    trenchwise_dfs = {}
    labelwise_dfs = {}
    for trench_idx in take(2, trench_idxs):
        trench_images = _get_trench_crops(images, trench_idx, uls, lrs)
        trench_trenchwise_df, trench_labelwise_df = _analyze_trench(
            trench_images,
            channels=channels,
            trenchwise_funcs=trenchwise_funcs,
            labelwise_funcs=labelwise_funcs,
        )
        trenchwise_dfs[(*frame_idx, t, trench_idx)] = trench_trenchwise_df
        labelwise_dfs[(*frame_idx, t, trench_idx)] = trench_labelwise_df
    framewise_df = pd.concat({(*frame_idx, t): framewise_df}, axis=1).T
    trenchwise_df = pd.concat(trenchwise_dfs, axis=1).T
    labelwise_df = pd.concat(labelwise_dfs, axis=0)
    framewise_df.index.names = [
        "filename",
        "position",
        "t",
        *framewise_df.index.names[3:],
    ]
    for df in (trenchwise_df, labelwise_df):
        df.index.names = ["filename", "position", "t", "trench", *df.index.names[4:]]
    return framewise_df, trenchwise_df, labelwise_df


def analyze_frames_and_trenches(selected_trenches, all_frames, func):
    all_frames_t = all_frames.groupby(["filename", "position"])
    res = []
    for frame_idx, trenches in take(
        2, util.iter_index(selected_trenches.groupby(["filename", "position"]))
    ):
        fp_frames = all_frames_t.get_group((frame_idx.filename, frame_idx.position))
        for t, frames in take(2, fp_frames.groupby("t")):
            res.append(func(frame_idx, t, trenches))
    return [
        pd.concat(filter(lambda x: x is not None, dfs), axis=0) for dfs in zip(*res)
    ]


labelwise_funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.1", "p0.3", "p0.5", "p0.7", "p0.9"): partial(
        np.percentile, q=(0.1, 0.3, 0.5, 0.7, 0.9)
    ),
}
trenchwise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}
framewise_funcs = {"sharpness": image.sharpness, **labelwise_funcs}

analyze_channels = ["MCHERRY", "YFP"]

res0 = analyze_frames_and_trenches(
    selected_trenches_segmentation,
    all_frames,
    partial(
        analyze_trenches,
        channels=analyze_channels,
        framewise_funcs=framewise_funcs,
        trenchwise_funcs=trenchwise_funcs,
        labelwise_funcs=labelwise_funcs,
    ),
)

# display(trenchwise_df)

In [ ]:
res0[2].tail()

In [ ]:
len(res0)

In [ ]:
res0[1].shape

In [ ]:
res0[1]

In [ ]:
selected_trenches_segmentation.head(10)

# Old segmentation

In [ ]:
trench_bboxes[("info", "hough_value")].plot.hist(bins=100)

In [ ]:
(trench_bboxes[("info", "hough_value")] > 90).sum() / len(trench_bboxes)

In [ ]:
selected_trenches_segmarker = trench_bboxes[
    trench_bboxes[("info", "hough_value")] > 90
].loc[IDX[:, :3, ["MCHERRY"], 0, :, :], :]
selected_trenches_index = next(iter(selected_trenches_segmarker.groupby("t")))[
    1
].index.droplevel("t")

In [ ]:
len(trench_bboxes) / len(selected_trenches_segmarker)

In [ ]:
selected_trenches_reporter = selected_trenches_segmarker.rename(
    index={"MCHERRY": "YFP"}, level="channel", copy=False
)

In [ ]:
selected_trenches_all = pd.concat(
    [selected_trenches_segmarker, selected_trenches_reporter]
).sort_index()

In [ ]:
[all_frames_future] = client.scatter([all_frames], broadcast=True)

In [ ]:
frame_stacks_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_all.groupby(["filename", "position"])
):
    frame_stacks_futures[frame_idx] = client.submit(
        workflow.get_trench_stacks, trenches, all_frames_future, image_limits
    )

In [ ]:
def do_segment_trench(img_stack):
    label_stack = np.stack([segment_trench(img) for img in img_stack])
    return zarr.array(
        label_stack,
        compressor=Blosc(cname="zstd", clevel=5, shuffle=Blosc.NOSHUFFLE, blocksize=0),
    )

In [ ]:
frame_segs_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_segs_futures[frame_idx] = client.submit(
        partial(workflow.map_trenchwise, do_segment_trench),
        frame_stacks_futures[frame_idx],
        trenches,
    )
# frame_segs_futures = valmap(partial(client.submit, partial(valmap, do_segment_trench)), frame_stacks_futures)

In [ ]:
_frame_stacks2 = workflow.get_trench_stacks(
    selected_trenches_all.xs(
        ("/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2", 0), drop_level=False
    ),
    all_frames,
    image_limits,
)

In [ ]:
partial(map_trenchwise, compute_regionprops, channels=["YFP"])(_frame)

In [ ]:
frame_regionprops_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_regionprops_futures[frame_idx] = client.submit(
        partial(map_trenchwise, compute_regionprops, channels=["YFP"]),
        frame_stacks_futures[frame_idx],
        trenches,
    )

In [ ]:
frame_regionprops = util.apply_map_futures(
    client.gather, frame_regionprops_futures, predicate=lambda x: x.status == "error"
)

In [ ]:
frame_segs = util.apply_map_futures(
    client.gather, frame_segs_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
a = util.get_one(util.get_one(frame_segs))

In [ ]:
plt.imshow(a[0])

In [ ]:
%%time
segs, seg_diags, seg_errs = zip(
    *[
        diagnostics.wrap_diagnostics(trench_segmentation.watershed.segment_trench)(img)
        for img in img_stack
    ]
)

# Segmentation testing

In [ ]:
%timeit map_stack({'focus': image.sharpness, ('p0.1','p0.5','p0.9'): partial(np.percentile, q=(0.1,0.5,0.9))}, _frame_stacks[_key])

In [ ]:
_frame_stacks[_key]

In [ ]:
funcs = {
    "mean": np.mean,
    "min": np.min,
    "max": np.max,
    ("p0.1", "p0.5", "p0.9"): partial(np.percentile, q=(0.1, 0.5, 0.9)),
}
a = map_stack_over_labels(funcs, _seg_masks[_key], _frame_stacks[_key])

In [ ]:
a.loc[IDX[:, 1:], :]

In [ ]:
map_frame_over_labels(
    {
        "mean": np.mean,
        "min": np.min,
        "max": np.max,
        ("p0.1", "p0.5", "p0.9"): partial(np.percentile, q=(0.1, 0.5, 0.9)),
    },
    _seg_masks[_key],
    _frame_stacks[_key],
)

In [ ]:
map_frame_over_labels(
    {
        "mean": np.mean,
        "min": np.min,
        "max": np.max,
        ("p0.1", "p0.5", "p0.9"): partial(np.percentile, q=(0.1, 0.5, 0.9)),
    },
    l0,
    i0,
)

In [ ]:
%timeit map_frame_over_labels({'mean': np.mean, 'min': np.min, 'max': np.max, ('p0.1','p0.5','p0.9'): partial(np.percentile, q=(0.1,0.5,0.9))}, l0, i0)

In [ ]:
%timeit map_frame_over_labels({'mean': np.mean, 'min': np.min, 'max': np.max}, l0, i0)

In [ ]:
%timeit pd.DataFrame({'label': l0.ravel(), 'value': i0.ravel()}).groupby('label').agg(['mean', 'min', 'max'])

In [ ]:
pd.DataFrame({"label": l0.ravel(), "value": i0.ravel()}).groupby("label").agg(
    ["mean", "min", "max", partial(np.percentile, q=(0.1, 0.5, 0.9))]
)

In [ ]:
l0 = _seg_masks[_key][0]
i0 = _frame_stacks[_key][0]

In [ ]:
%%timeit
for l in range(l0.max()):
    i0[l0 == l].mean()

In [ ]:
%timeit numpy_indexed.group_by(l0.ravel(), i0.ravel(), reduction=np.mean)

In [ ]:
_rps = compute_regionprops(
    _seg_masks[_key], _frame_stacks[_key]
)  # , _frame_stacks[_key._replace(channel='YFP')])
_rps.head()

In [ ]:
_trenches_segmarker = selected_trenches_segmarker.loc[
    IDX[["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2"]], :
].iloc[10:12]
_trenches_segmarker_index = util.get_one(_trenches_segmarker.groupby("t"))[
    1
].index.droplevel("t")
_trenches_reporter = _trenches_segmarker.rename(
    index={"MCHERRY": "YFP"}, level="channel", copy=False
)
_trenches_all = pd.concat([_trenches_segmarker, _trenches_reporter]).sort_index()
_trenches_all

In [ ]:
_frame_stacks = workflow.get_trench_stacks(_trenches_all, all_frames, image_limits)

In [ ]:
_seg_masks = map_trenchwise(do_segment_trench, _frame_stacks, _trenches_segmarker_index)

In [ ]:
_seg_masks

In [ ]:
_key = util.get_one(_seg_masks.keys())
_key

In [ ]:
_rps = compute_regionprops(
    _seg_masks[_key], _frame_stacks[_key]
)  # , _frame_stacks[_key._replace(channel='YFP')])

In [ ]:
from trench_segmentation.watershed import _trench_img

In [ ]:
%%time
segment_trench_diag = diagnostics.wrap_diagnostics_stack(
    trench_segmentation.watershed.segment_trench
)
segs, seg_diags, seg_errs = segment_trench_diag(util.get_one(_frame_stacks))

In [ ]:
ui.show_plot_stack(seg_diags)

In [ ]:
a = seg_diags[31]["img_k1_frangi"].data
b = seg_diags[31]["img"].data
c = seg_diags[31]["img_k1"].data

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(c)
plt.figure(figsize=(12, 12))
plt.imshow(skimage.filters.rank.gradient(c, skimage.morphology.disk(1.5)))
plt.figure(figsize=(12, 12))
plt.imshow(skimage.filters.rank.percentile(c, skimage.morphology.disk(3), p0=0.7))
plt.figure(figsize=(12, 12))
plt.imshow(
    skimage.filters.rank.mean_bilateral(c, skimage.morphology.disk(3), s0=5, s1=10)
)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(a)
plt.figure(figsize=(12, 12))
# plt.imshow(skimage.filters.rank.gradient(a, skimage.morphology.disk(5)))
plt.imshow(b)
plt.figure(figsize=(12, 12))
plt.imshow(skimage.filters.rank.median(b, skimage.morphology.disk(2)))
plt.figure(figsize=(12, 12))
plt.imshow(skimage.filters.gaussian(a, 1.5))

### Ref

In [ ]:
frame_stacks_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_all.groupby(["filename", "position"])
):
    frame_stacks_futures[frame_idx] = client.submit(
        workflow.get_trench_stacks, trenches, all_frames_future, image_limits
    )

In [ ]:
frame_segs_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_segs_futures[frame_idx] = client.submit(
        partial(map_trenchwise, do_segment_trench),
        frame_stacks_futures[frame_idx],
        trenches,
    )
# frame_segs_futures = valmap(partial(client.submit, partial(valmap, do_segment_trench)), frame_stacks_futures)

In [ ]:
frame_regionprops_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_regionprops_futures[frame_idx] = client.submit(
        partial(map_trenchwise, compute_regionprops, channels=["YFP"]),
        frame_stacks_futures[frame_idx],
        trenches,
    )

In [ ]:
_rps = compute_regionprops(
    _seg_masks[_key], _frame_stacks[_key]
)  # , _frame_stacks[_key._replace(channel='YFP')])

In [ ]:
partial(map_trenchwise, compute_regionprops, channels=["YFP"])(_frame)

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)